# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [19]:
#Your code here

import requests
import pandas as pd
import json

term = 'Whiskey bars'
location = 'NYC'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'
api_key = '77BFSSmE-PI0wJdeJMd9WbHkPqSC6fOXwTkPoe6hQid4M4heWgRZDhJvPB9LQQe1CO5kFYB-PJi0cQ2ImGDYEI5sCqpFOY_VjInzhTiPFgkfkVYync5J_oI_IAORXHYx'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
parameters = {
    'term': term.replace(' ', '+'),
    'location': location.replace(' ', '+')
}

data = requests.get(url, headers = headers, params = parameters)

raw_data = data.json()

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [16]:
# Your code here; use a function or loop to retrieve all the results from your original request
raw_data

{'businesses': [{'id': 'c3Ww3pgeHjBi3ctkeUWrjQ',
   'alias': 'whiskey-tavern-new-york',
   'name': 'Whiskey Tavern',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/P3eezkfdJ1cP3Z34T5Kmyg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/whiskey-tavern-new-york?adjust_creative=87VyFdZxZtBJg6MjrR8jQQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=87VyFdZxZtBJg6MjrR8jQQ',
   'review_count': 743,
   'categories': [{'alias': 'whiskeybars', 'title': 'Whiskey Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.71646, 'longitude': -73.99969},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '79 Baxter St',
    'address2': '',
    'address3': '',
    'city': 'New York',
    'zip_code': '10013',
    'country': 'US',
    'state': 'NY',
    'display_address': ['79 Baxter St', 'New York, NY 10013']},
   'phone': '+12123749119',
   'display_phone': '(212) 374-9119',
   'distance': 1307.2374759358268},
  {'id': 'ajdv6_mWIk

In [23]:
all_results = []
for data in raw_data['businesses']:
    all_results.append(data)

df = pd.DataFrame(all_results)

In [31]:
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,whiskey-tavern-new-york,"[{'alias': 'whiskeybars', 'title': 'Whiskey Ba...","{'latitude': 40.71646, 'longitude': -73.99969}",(212) 374-9119,1307.237476,c3Ww3pgeHjBi3ctkeUWrjQ,https://s3-media1.fl.yelpcdn.com/bphoto/P3eezk...,False,"{'address1': '79 Baxter St', 'address2': '', '...",Whiskey Tavern,+12123749119,$$,4.5,743,[],https://www.yelp.com/biz/whiskey-tavern-new-yo...
1,copper-and-oak-new-york-5,"[{'alias': 'bars', 'title': 'Bars'}]","{'latitude': 40.721065, 'longitude': -73.98986}",(212) 460-5546,1776.962415,ajdv6_mWIkAXzn71CUdpkw,https://s3-media2.fl.yelpcdn.com/bphoto/9ZFmNH...,False,"{'address1': '157 Allen St', 'address2': '', '...",Copper & Oak,+12124605546,$$$,4.5,88,[],https://www.yelp.com/biz/copper-and-oak-new-yo...
2,brandy-library-new-york,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.720073, 'longitude': -74.007227}",(212) 226-5545,1958.084250,RW-v9cm9rQZS9cSihuXfvw,https://s3-media4.fl.yelpcdn.com/bphoto/745pwD...,False,"{'address1': '25 N Moore St', 'address2': None...",Brandy Library,+12122265545,$$$,4.0,564,[restaurant_reservation],https://www.yelp.com/biz/brandy-library-new-yo...
3,dutch-kills-long-island-city,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...","{'latitude': 40.747615, 'longitude': -73.940159}",(718) 383-2724,6541.943252,3lLBRLbXAIgd1yZgWgnfeg,https://s3-media1.fl.yelpcdn.com/bphoto/I4paO4...,False,"{'address1': '27-24 Jackson Ave', 'address2': ...",Dutch Kills,+17183832724,$$,4.0,715,[],https://www.yelp.com/biz/dutch-kills-long-isla...
4,on-the-rocks-new-york-2,"[{'alias': 'lounges', 'title': 'Lounges'}]","{'latitude': 40.76373, 'longitude': -73.99212}",(212) 247-2055,6481.155010,9JGdwVMF7MW452eGmCpnDw,https://s3-media3.fl.yelpcdn.com/bphoto/SFFl_L...,False,"{'address1': '696 10th Ave', 'address2': '', '...",On The Rocks,+12122472055,$$$,4.0,230,[],https://www.yelp.com/biz/on-the-rocks-new-york...


## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [36]:
#Your code here
for data in df:
    for max_vals in df['review_count']:
        print()

1449

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!